In [1]:
import sqlalchemy
import pandas as pd

In [ ]:
# 1. get user_id, user_created_at,  json_outuput, decision_at, partner for rejected
# 2. get all json output together into list with user_id and reject reasons, 
# 3. concat reject reason in one row itself.

In [3]:
users = """
    select user_id from user_registeration_coupon_code_details where coupon_code in 
    ('ESSMART','roinet','gramophone','yodacart','buyume')
"""
f1, b2b_users, error_message = data_fetch_v2(users, host = "13.126.97.63")


In [7]:
users

"\n    select user_id from user_registeration_coupon_code_details where coupon_code in \n    ('ESSMART','roinet','gramophone','yodacart','buyume')\n"

In [4]:
reject_json = """
    select user_id, json_extract(output_json, '$.rejection_reason') as reject_rsn from de_looger 
   where purpose = 'bureaucheck' 
    and json_extract(output_json, '$.decision.approved') = 0 
    and user_id in {tuple(b2b_users['user_id'])}
"""
f1, b2b_reject_reason, error_message = data_fetch_v2(reject_json, host = "13.126.97.63")


(pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '{tuple(b2b_users['user_id'])}' at line 4")
[SQL: 
    select user_id, json_extract(output_json, '$.rejection_reason') as reject_rsn from de_looger 
   where purpose = 'bureaucheck' 
    and json_extract(output_json, '$.decision.approved') = 0 
    and user_id in {tuple(b2b_users['user_id'])}
]
(Background on this error at: https://sqlalche.me/e/20/f405)


In [38]:
user_ids = tuple(b2b_users['user_id'])  # Convert user IDs to tuple

# Ensure the user IDs are not empty
if not user_ids:
    user_ids = (None,)  # Fallback for empty tuples to avoid syntax errors

# Format the query
reject_json = f"""
    SELECT user_id, 
           json_extract(output_json, '$.rejection_reason') AS reject_rsn 
    FROM de_looger 
    WHERE purpose = 'bureaucheck' 
      AND json_extract(output_json, '$.decision.approved') = 0 
      AND user_id IN {user_ids}
"""

# Call data_fetch_v2
f1, b2b_reject_reason, error_message = data_fetch_v2(reject_json, host="13.126.97.63")


In [6]:
b2b_reject_reason

""


In [41]:
b2b_reject_reason_data = b2b_reject_reason.to_dict(orient="records")

In [8]:
b2b_reject_reason_data

NameError: name 'b2b_reject_reason_data' is not defined

In [ ]:
#put it in KVP form,remvoing text user_id and reject_rsn
data = {items["user_id"]: items["reject_rsn"] for items in b2b_reject_reason_data}
data

In [49]:
import json

In [59]:
# for k, v in data.items():
#     reject_messages = json.loads(v)
#     for v in reject_messages:
#         print(v['message'])


rows =[]
for k, v in data.items():
    reject_messages = json.loads(v)
    for val in reject_messages:
        rows.append({'user_id':k,"reason":val['message']})
     
print(rows) 

[{'user_id': '206cf27c-aaf4-4a37-a7d1-c9f30106850c', 'reason': 'user_and_lead_phone_mismatch'}, {'user_id': 'e25df091-4299-45f5-8d99-54de4ff3b337', 'reason': 'user_and_lead_phone_mismatch'}, {'user_id': '3244d992-8691-4fb1-88a8-2a2d9fced9f5', 'reason': '0 < age < 20 or age > 55'}, {'user_id': '3244d992-8691-4fb1-88a8-2a2d9fced9f5', 'reason': 'user_and_lead_phone_mismatch'}, {'user_id': '7447d9a7-075a-40de-9448-4dc1b13f1aa2', 'reason': '0 < age < 20 or age > 55'}, {'user_id': '7447d9a7-075a-40de-9448-4dc1b13f1aa2', 'reason': 'user_and_lead_phone_mismatch'}, {'user_id': 'c1c48e87-1f6b-479e-a5a3-81afc1d56e7e', 'reason': '0 < age < 20 or age > 55'}, {'user_id': '354094cf-6f75-4de5-b975-fd9cb73a15c6', 'reason': 'approved_lead_does_not_exists'}, {'user_id': '354094cf-6f75-4de5-b975-fd9cb73a15c6', 'reason': 'pre_qualified_and_lead_pan_mismatch'}, {'user_id': '354094cf-6f75-4de5-b975-fd9cb73a15c6', 'reason': 'user_and_lead_phone_mismatch'}, {'user_id': '8eb8d3e8-49f3-4a72-864c-5599cb291654', '

In [60]:
df = pd.DataFrame(rows)
df

,user_id,reason
0,206cf27c-aaf4-4a37-a7d1-c9f30106850c,user_and_lead_phone_mismatch
1,e25df091-4299-45f5-8d99-54de4ff3b337,user_and_lead_phone_mismatch
2,3244d992-8691-4fb1-88a8-2a2d9fced9f5,0 < age < 20 or age > 55
3,3244d992-8691-4fb1-88a8-2a2d9fced9f5,user_and_lead_phone_mismatch
4,7447d9a7-075a-40de-9448-4dc1b13f1aa2,0 < age < 20 or age > 55
5,7447d9a7-075a-40de-9448-4dc1b13f1aa2,user_and_lead_phone_mismatch
6,c1c48e87-1f6b-479e-a5a3-81afc1d56e7e,0 < age < 20 or age > 55
7,354094cf-6f75-4de5-b975-fd9cb73a15c6,approved_lead_does_not_exists
8,354094cf-6f75-4de5-b975-fd9cb73a15c6,pre_qualified_and_lead_pan_mismatch
9,354094cf-6f75-4de5-b975-fd9cb73a15c6,user_and_lead_phone_mismatch


In [61]:
df.groupby('reason').count()

,user_id
reason,
0 < age < 20 or age > 55,3
approved_lead_does_not_exists,9
pre_qualified_and_lead_pan_mismatch,9
user_and_lead_phone_mismatch,13


In [ ]:
#steps.
1. import sqlalchemy, pandas, json
2. create a df from mysql, to filter out b2b users
3. futher get json reject and rejected user id from step 2 users
4. 
df = user_id, reject
convert the whole df to dict, orient ias records. 
5. 